In [421]:
#-------------------------------
#Funktionen:
#-------------------------------

# Print iterations progress
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [422]:
import numpy as np
from osgeo import gdal
from osgeo import osr
import time

#-------------------------------
#File einlesen:
#-------------------------------
fobj = open('Daten/Waldpunktwolke_1000.txt', "r") #Input File

pointlist = []

next(fobj)
for line in fobj:
    line = line.strip()
    line= line.split("\t")   
    pointlist.append(line)
fobj.close

<function TextIOWrapper.close()>

In [423]:
#-------------------------------
#Daten einlesen und nach Typ Filtern:
#-------------------------------

pointarray = np.array(pointlist) 
pointarray = pointarray.astype(float)  
print(len(pointarray))

#Neuer Array ohne Gebauede und Boden, Klassen 3, 4, 5
vegetationslist = []
bodenlist=[]

print("Vegetations-und Bodenliste einlesen:")
printProgressBar(0, len(pointarray), prefix = 'Progress:', suffix = 'Complete', length = 50)
j=0
for i in pointarray:
    printProgressBar(j+1, len(pointarray), prefix = 'Progress:', suffix = 'Complete', length = 50)
    j+=1
    if i[6] == 3 or i[6] ==4 or i[6] ==5:
        vegetationslist.append(i)
    if i[6] == 2:
        bodenlist.append(i)

vegarray = np.array(vegetationslist)        
bodenarray = np.array(bodenlist)


#----------------------------------
#Extend Array with random numbers:
#----------------------------------
#max values
maxvals = np.amax(pointarray,axis=0) 
xmax=maxvals[0]
ymax=maxvals[1]
# print ("xmax:",xmax)
# print ("ymax:",ymax)

#min values
minvals = np.amin(pointarray,axis=0) 
xmin=minvals[0]
ymin=minvals[1]
# print ("xmin:",xmin)
# print ("ymin:",ymin)

#empty array with extent dimensions
nrows= int(np.ceil(ymax-ymin))
ncols= int(np.ceil(xmax-xmin))

bodencountout_array = np.empty((nrows,ncols))
bodencountout_array.fill(0)

bodenhoehenout_array = np.empty((nrows,ncols))
bodenhoehenout_array.fill(0)

vegcountout_array = np.empty((nrows,ncols))
vegcountout_array.fill(0)

veghoeheout_array = np.empty((nrows,ncols))
veghoeheout_array.fill(0)

vegstdout_array = np.empty((nrows,ncols))
vegstdout_array.fill(0)


1000
Vegetations-und Bodenliste einlesen:
Progress: |██████████████████████████████████████████████████| 100.0% Complete


In [424]:
#----------------------------------
#Bodenpunkte zaehlen:
#----------------------------------
bodencount = 0
countlist =[]
arraycount_x = 0
arraycount_y = 0
bodenhoehenlist = []

print("Bodenpunkte zuordnen:")
printProgressBar(0, len(np.arange(ymax,ymax-nrows,-1)), prefix = 'Progress:', suffix = 'Complete', length = 50)
j=0
for y in np.arange(ymax,ymax-nrows,-1):
    printProgressBar(j+1, len(np.arange(ymax,ymax-nrows,-1)), prefix = 'Progress:', suffix = 'Complete', length = 50)
    j+=1
    # print("yRichtung:",y)
    # print("ArraypositionY:",arraycount_y)
    for x in np.arange(xmin,xmin+ncols,1):
        # print("xRichtung:",x)
        # print("ArraypositionX:",arraycount_x)
        for i in bodenarray:
            if x <= i[0] < x+1 and y >= i[1] > y-1:
                bodencount+=1
                bodenhoehenlist.append(i[2])
        bodencountout_array[arraycount_y][arraycount_x]=bodencount
        bodenhoehenout_array[arraycount_y][arraycount_x]=np.mean(bodenhoehenlist)
        arraycount_x+=1     
        bodencount=0
        bodenhoehenlist=[]
    arraycount_x=0
    arraycount_y+=1
    # print("xRichtungEnde")
    #bodenhoehenout_array[np.isnan(bodenhoehenout_array)] = 0




Bodenpunkte zuordnen:
Progress: |██████████████████████████████████████████████████| 100.0% Complete
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [425]:
#----------------------------------
#Mittlere Hoehe der VegPunkte:
#----------------------------------
vegcount = 0
countlist =[]
arraycount_x = 0
arraycount_y = 0
veghoehenlist = []

print("VegPunkte zuordnen:")
printProgressBar(0, len(np.arange(ymax,ymax-nrows,-1)), prefix = 'Progress:', suffix = 'Complete', length = 50)
j=0
for y in np.arange(ymax,ymax-nrows,-1):
    printProgressBar(j+1, len(np.arange(ymax,ymax-nrows,-1)), prefix = 'Progress:', suffix = 'Complete', length = 50)
    j+=1
    # print("yRichtung:",y)
    # print("ArraypositionY:",arraycount_y)
    for x in np.arange(xmin,xmin+ncols,1):
        # print("xRichtung:",x)
        # print("ArraypositionX:",arraycount_x)
        for i in vegarray:
            if x <= i[0] < x+1 and y >= i[1] > y-1: #Hier sind wir in der jeweiligen Zelle
                veghoehenlist.append(i[2])
                vegcount+=1

        veghoeheout_array[arraycount_y][arraycount_x]=np.mean(veghoehenlist) #Mittlere Hoehe der VegPunkte
        vegstdout_array[arraycount_y][arraycount_x]=np.std(veghoehenlist) #Standardabweichung der VegPunkte
        vegcountout_array[arraycount_y][arraycount_x]=vegcount #Anzahl der Vegetationspunkte in Zelle
        arraycount_x+=1     
        vegcount=0
        veghoehenlist=[]
    arraycount_x=0
    arraycount_y+=1
    # print("xRichtungEnde") 
    #vegstdout_array[np.isnan(vegstdout_array)] = 0
    #veghoeheout_array[np.isnan(veghoeheout_array)] = 0

VegPunkte zuordnen:
Progress: |██████████████████████████████████████████████████| 100.0% Complete
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/core/_methods.py:127: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [426]:
#-------------------------------
#Tatsaechliche VegHoehe:
#-------------------------------
vegtathoehe_array = np.empty((nrows,ncols))
vegtathoehe_array = veghoeheout_array - bodenhoehenout_array


In [427]:
#-------------------------------
#OutRaster schreiben:
#-------------------------------


#Boden Raster
driver = gdal.GetDriverByName("GTiff")
dataset = driver.Create("Export/BodenCount_1000.tif", ncols, nrows, 1, gdal.GDT_Float32)
dataset.SetGeoTransform((xmin,1,0,ymax,0,-1))

#Koordinatensystem definieren:
dstSRS = osr.SpatialReference()
dstSRS.ImportFromEPSG(32632)
dest_wkt = dstSRS.ExportToWkt()

dataset.SetProjection(dest_wkt)

#Raster ausgeben:
bandout = dataset.GetRasterBand(1).WriteArray(bodencountout_array)
dataset.FlushCache()

In [428]:
#Bodenhoehen Raster
driver = gdal.GetDriverByName("GTiff")
dataset = driver.Create("Export/BodenHoehen_1000.tif", ncols, nrows, 1, gdal.GDT_Float32)
dataset.SetGeoTransform((xmin,1,0,ymax,0,-1))

#Koordinatensystem definieren:
dstSRS = osr.SpatialReference()
dstSRS.ImportFromEPSG(32632)
dest_wkt = dstSRS.ExportToWkt()

dataset.SetProjection(dest_wkt)

#Raster ausgeben:
bandout = dataset.GetRasterBand(1).WriteArray(bodenhoehenout_array)
dataset.FlushCache()

In [429]:

#Vegetation Zaehler Raster
driver = gdal.GetDriverByName("GTiff")
dataset = driver.Create("Export/VegCount_1000.tif", ncols, nrows, 1, gdal.GDT_Float32)
dataset.SetGeoTransform((xmin,1,0,ymax,0,-1))

#Koordinatensystem definieren:
dstSRS = osr.SpatialReference()
dstSRS.ImportFromEPSG(32632)
dest_wkt = dstSRS.ExportToWkt()

dataset.SetProjection(dest_wkt)

#Raster ausgeben:
bandout = dataset.GetRasterBand(1).WriteArray(vegcountout_array)
dataset.FlushCache()

In [430]:

#Vegetation Mittlere Hoehe Raster
driver = gdal.GetDriverByName("GTiff")
dataset = driver.Create("Export/VegHoehen_1000.tif", ncols, nrows, 1, gdal.GDT_Float32)
dataset.SetGeoTransform((xmin,1,0,ymax,0,-1))

#Koordinatensystem definieren:
dstSRS = osr.SpatialReference()
dstSRS.ImportFromEPSG(32632)
dest_wkt = dstSRS.ExportToWkt()

dataset.SetProjection(dest_wkt)

#Raster ausgeben:
bandout = dataset.GetRasterBand(1).WriteArray(vegtathoehe_array)
dataset.FlushCache()

In [431]:

#Vegetation Standardabweichung Hoehe Raster
driver = gdal.GetDriverByName("GTiff")
dataset = driver.Create("Export/VegStd_1000.tif", ncols, nrows, 1, gdal.GDT_Float32)
dataset.SetGeoTransform((xmin,1,0,ymax,0,-1))

#Koordinatensystem definieren:
dstSRS = osr.SpatialReference()
dstSRS.ImportFromEPSG(32632)
dest_wkt = dstSRS.ExportToWkt()

dataset.SetProjection(dest_wkt)

#Raster ausgeben:
bandout = dataset.GetRasterBand(1).WriteArray(vegstdout_array)
dataset.FlushCache()

In [432]:
np.savetxt(testout.txt,vegstdout_array, delimiter=' ')

NameError: name 'testout' is not defined